In [52]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.0 MB/s eta 0:00:0000:010:01
  Using cached regex-2022.6.2-cp39-cp39-macosx_11_0_arm64.whl (281 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [8]:
import pandas as pd

data = pd.read_excel (r'data.xlsx') 
df = pd.DataFrame(data)
df.head()

,Unnamed: 0,CASE,Q4 ANSWER,Q4 Score,Q9 ANSWER,Q9 Score,Score 1-4 Q4,Ashray Score,Score 1-4 Q9,Ashray Score.1
0,NaN,Case 2a,"Yes, offering the discounts will help increase...",44,"In a normal scenario, the client would pay 100...",44,2,NaN,2.0,2.0
1,NaN,Case 2a,It depends. If they want to promote the use of...,33,If for every $10 increase the possibility of a...,78,2,NaN,4.0,2.0
2,NaN,Case 2a,the introduction is very profitable from Vivit...,0,the increase in premiums corresponds to an inc...,33,1,NaN,2.0,4.0
3,NaN,Case 2a,"To decide whether this is a good idea, I would...",0,The expected cost increase by 150$ per opt-out...,44,1,NaN,2.0,1.0
4,NaN,Case 2a,Ich würde empfehlen das Angebot auch den beste...,6,`-,0,1,NaN,1.0,1.0


In [9]:
df.columns

Index(['Unnamed: 0', 'CASE', 'Q4 ANSWER', 'Q4 Score', 'Q9 ANSWER', 'Q9 Score',
       'Score 1-4 Q4', 'Ashray Score', 'Score 1-4 Q9', 'Ashray Score.1'],
      dtype='object')

In [76]:
q4_df = df[["Q4 ANSWER","Q4 Score","Score 1-4 Q4"]]
q4_df.head()

,Q4 ANSWER,Q4 Score,Score 1-4 Q4
0,"Yes, offering the discounts will help increase...",44,2
1,It depends. If they want to promote the use of...,33,2
2,the introduction is very profitable from Vivit...,0,1
3,"To decide whether this is a good idea, I would...",0,1
4,Ich würde empfehlen das Angebot auch den beste...,6,1


In [77]:
q9_df = df[["Q9 ANSWER","Q9 Score","Score 1-4 Q9"]]
q9_df.head()

,Q9 ANSWER,Q9 Score,Score 1-4 Q9
0,"In a normal scenario, the client would pay 100...",44,2.0
1,If for every $10 increase the possibility of a...,78,4.0
2,the increase in premiums corresponds to an inc...,33,2.0
3,The expected cost increase by 150$ per opt-out...,44,2.0
4,`-,0,1.0


In [78]:
q4_df = q4_df.rename(columns = {"Q4 ANSWER":"answer","Q4 Score":"score_100","Score 1-4 Q4":"score_4"})
q9_df = q9_df.rename(columns = {"Q9 ANSWER":"answer","Q9 Score":"score_100","Score 1-4 Q9":"score_4"})

In [79]:
q4_df.head()

,answer,score_100,score_4
0,"Yes, offering the discounts will help increase...",44,2
1,It depends. If they want to promote the use of...,33,2
2,the introduction is very profitable from Vivit...,0,1
3,"To decide whether this is a good idea, I would...",0,1
4,Ich würde empfehlen das Angebot auch den beste...,6,1


In [80]:
q4_df = q4_df[q4_df.answer != "`-"]
q9_df = q9_df[q9_df.answer != "`-"]

In [81]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /Users/uday/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
# function to clean data
def clean_data(df, col, clean_col):

    # change to lower and remove spaces on either side
    df[clean_col] = df[col].apply(lambda x: x.lower().strip())

    # remove extra spaces in between
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(' +', ' ', x))

    # remove punctuation
    df[clean_col] = df[clean_col].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

    # remove stopwords and get the stem
    df[clean_col] = df[clean_col].apply(lambda x: ' '.join(st.stem(text) for text in x.split() if text not in stop_words))

    return df

In [83]:
q4_df = clean_data(q4_df, 'answer', 'answer')
q9_df = clean_data(q9_df, 'answer', 'answer')

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
v = TfidfVectorizer()

In [91]:
x = v.fit_transform(q4_df['answer'])
q4_df['tfidf'] = list(x.toarray())

weights = np.asarray(x.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': v.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,weight
200,custom,0.096410
986,would,0.074586
246,discount,0.070932
323,exist,0.064906
605,offer,0.064096
689,profit,0.053185
149,claim,0.053035
445,increas,0.049139
946,vivita,0.047082
219,decreas,0.041537


In [92]:
x = v.fit_transform(q9_df['answer'])
q9_df['tfidf'] = list(x.toarray())

weights = np.asarray(x.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': v.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,weight
211,increas,0.161797
62,claim,0.092335
335,premium,0.086447
302,opt,0.079508
345,profit,0.079015
489,would,0.072301
90,cost,0.069526
383,revenu,0.056382
197,higher,0.055100
108,decreas,0.053639


In [182]:
q9_df.head()

,answer,score_100,score_4,tfidf
0,normal scenario client would pay would die rec...,44,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,everi increas possibl claim increas eventu cos...,78,4.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,increas premium correspond increas meanwhil in...,33,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,expect cost increas per opt wherea revenu increas,44,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,probabl claim increas possibl cost also increa...,67,3.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [154]:
import seaborn as sns
from collections import defaultdict
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline

from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV     
from sklearn import linear_model

In [190]:
# insistent our storage 
bench_mark = defaultdict(list)

# split data random state 0 and test_size 0.25 default as you did not give the test_size

#X_train, X_test, y_train, y_test = train_test_split(q4_df['answer'], q4_df['score_4'], random_state = 0, test_size = 0.1)
# X_train, X_test, y_train, y_test = train_test_split(q4_df['answer'], q4_df['score_4'], random_state = 0, test_size = 0.1)
X_train, X_test, y_train, y_test = train_test_split(q9_df['answer'], q9_df['score_100'], random_state = 0, test_size = 0.1)
# X_train, X_test, y_train, y_test = train_test_split(q9_df['answer'], q9_df['score_100'], random_state = 0, test_size = 0.1)


#get the transformer params
tfidf_params = dict(sublinear_tf= True, 
                       min_df = 1,
                       norm= 'l2', 
                       ngram_range= (1,2), 
                       stop_words ='english')

# list of models we would like to complete 
models = [
#LogisticRegressionCV(cv=5,solver='saga',scoring='accuracy', random_state=1, n_jobs=-1),
LinearRegression(),
linear_model.Ridge(alpha=.5),
linear_model.Lasso(alpha=0.1),
#linear_model.LassoLars(alpha=.1, normalize=False),
#linear_model.BayesianRidge()
]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test)
print(q9_df['answer'].shape == q9_df['score_100'].shape)

(176,) (20,) (176,) (20,)
108     0
14      0
43     44
162     0
263    25
206    42
248    50
191     0
95     56
23     67
9      11
7      78
192    67
246    83
262     0
278     0
190    67
77     78
57     56
214     0
Name: score_100, dtype: int64
True


In [191]:
# train, test and store each model 
for model in models:

    # our pipeline is changed to accept model
    clf = Pipeline(steps=[
        ('features', TfidfVectorizer(**tfidf_params)),
        ('model', model) #just model not model() as we have done that in models list
    ])

    clf.fit(X_train,y_train)
    score = clf.score(X_test,y_test)

    model_name = clf.named_steps['model'].__class__.__name__ # hack to get name

    model_params = clf.named_steps['model']. get_params()


    print(f'{model_name} Scored: {score:.3f}\n')

    bench_mark['model_name'].append(model_name)
    bench_mark['score'].append(score)
    bench_mark['model'].append(clf)
    bench_mark['used_params'].append(model_params)

# in the end, place the bench_mark to DataFrame
models_df = pd.DataFrame(bench_mark)


LinearRegression Scored: 0.523

Ridge Scored: 0.504

Lasso Scored: 0.455



In [192]:
print(list(y_test))

[0, 0, 44, 0, 25, 42, 50, 0, 56, 67, 11, 78, 67, 83, 0, 0, 67, 78, 56, 0]


In [193]:
# now you have the trained modes in DataFrame, their scores and parameters. 
#You can access and use any model.

linear_reg = models_df[models_df['model_name']=='LinearRegression']['model'].iloc[0]

y_preds = linear_reg.predict(X_test)

print(y_preds)

[21.16148264 24.69508571 29.52820747 28.34415586 36.13718005 39.99196985
 31.69359707 17.26149298 47.98174143 58.36504244 13.1502581  82.40464348
 17.68997079 80.13360267 25.52964189 54.07244302 77.77343561 74.49509305
 47.43098949 10.12185117]


In [194]:
linear_reg = models_df[models_df['model_name']=='Ridge']['model'].iloc[0]

y_preds = linear_reg.predict(X_test)
print(y_preds)

[25.94227429 29.28723925 33.71166646 31.05884632 40.00118157 41.81467556
 34.07090042 24.47092511 48.70075656 56.39986064 18.19676814 72.75633744
 28.74305197 69.9408796  29.94776635 52.05950618 69.53626136 66.28512588
 45.12586258 16.44291978]


In [195]:
linear_reg = models_df[models_df['model_name']=='Lasso']['model'].iloc[0]

y_preds = linear_reg.predict(X_test)
print(y_preds)

[11.56659477 22.64229647 42.43509219 44.13839448 57.92730671 51.95386457
 38.63437517 37.94497399 61.06622356 44.17965749 22.85366507 82.17779014
 27.24576589 64.92348456 20.15174529 17.95243658 87.64820445 72.30721986
 48.10113282 37.94497399]
